# Process data for MODEL 1

In [ ]:
from Melokai.dataReadFromDisk import *
df = load_df_from_csv()
# df = df[df["total_length"]<120]
df = df[(df["BeatDivisor"]==4) & (df["total_length"]<300)]
# df=df[:500]

In [ ]:
len(df)

In [ ]:
num_of_batch,input_audios,output_beatmaps,processed_beatmap_list,hyper_params_list = new_process_audio_and_beatmap_for_model_all_and_save(df)

In [6]:
# Mapping of (first_column, second_column) pair to specific index in 21-feature space
combination_map = {
    (0, 0): 0,

    (1, 0): 1, 
    (5, 0): 1, 

    (2, 0): 2, (2, 1): 3, (2, 2): 4,
    (6, 0): 2, (6, 1): 3, (6, 2): 4,

    (8, 0): 5, (8, 1): 6, (8,2):7,
    (12, 0): 5, (12, 1): 6, (12,2):7,

}

# Output list of 21-features
output_8_features = []

for op in output_beatmaps:
    temp = []
    
    # For each row in the op, create a 21-element array with zeros
    for row in op:
        feature_row = np.zeros(8, dtype=int)
        
        # Get the values of the first and second columns
        first_col, second_col = row[0], row[1]
        
        # Check if this (first_col, second_col) pair exists in the combination_map
        if (first_col, second_col) in combination_map:
            # Set the corresponding position to 1
            index = combination_map[(first_col, second_col)]
            feature_row[index] = 1
        
        # Append the feature row to temp
        temp.append(feature_row)
    
    # Convert temp to a numpy array and append to output_21_features
    output_8_features.append(np.array(temp))







In [7]:
input_audios_with_hyperparams = []
#scale hyper params 
#divide by [350,300,1,10,10,10,10,10,10,10,10,10]

for index, ia in enumerate(input_audios):
    temp = []
    normalizedHyperParams = [(hyper_params_list[index][0]/350),(hyper_params_list[index][1]/300),(hyper_params_list[index][2]/350),(hyper_params_list[index][3]/10),(hyper_params_list[index][4]/10),(hyper_params_list[index][5]/10),(hyper_params_list[index][6]/10),(hyper_params_list[index][7]/10),(hyper_params_list[index][8]/10),(hyper_params_list[index][9]/10),(hyper_params_list[index][10]/10)]
    for row in ia:
        new_row = np.concatenate((row, normalizedHyperParams))
        temp.append(new_row)

    input_audios_with_hyperparams.append(temp)

# MODEL TO TRAIN
on python 3.12.4


In [84]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Example variables
input_list = input_audios_with_hyperparams  # This is your list of song inputs, where each song is (num_beats, 13)
output_list = output_8_features  # This is your list of song outputs, where each song is (num_beats, 2)

# Parameters
num_input_features = 91  # 91 features per beat in input
num_output_features = 8  # 8 features to predict per beat

# Padding sequences (songs) to handle variable length songs
padded_inputs = pad_sequences(input_list, padding='post', dtype='float32')
padded_outputs = pad_sequences(output_list, padding='post', dtype='float32')

new

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Define Transformer block (same as before)
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, mask=None, training=False):
        attn_output = self.att(inputs, inputs, attention_mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


# Position Embedding Layer
class PositionEmbedding(layers.Layer):
    def __init__(self, maxlen, embed_dim):
        super(PositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.token_emb = layers.Embedding(input_dim=10000, output_dim=embed_dim)  # adjust input_dim based on vocabulary size
        self.maxlen = maxlen

    def call(self, x):
        maxlen = tf.shape(x)[-2]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return x + positions


# Custom model with masking and Transformer block
# In ModelWithMasking class
class ModelWithMasking(tf.keras.Model):
    def __init__(self, maxlen, embed_dim, num_heads, ff_dim, num_classes):
        super(ModelWithMasking, self).__init__()
        self.position_embedding = PositionEmbedding(maxlen, embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        self.output_layer = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Masking for padded inputs
        mask = tf.cast(tf.not_equal(tf.reduce_sum(inputs, axis=-1), 0), dtype=tf.float32)[:, :, tf.newaxis]
        
        x = self.position_embedding(inputs)
        x = self.transformer_block(x, mask=mask)
        # Remove the pooling layer to get sequence outputs
        return self.output_layer(x)  # This will output shape (batch_size, seq_len, num_classes)



# Example setup
maxlen_input = padded_inputs.shape[1]  # Length of padded input sequences
embed_dim = 91  # Embedding dimension (should match your input features)
num_heads = 4  # Number of attention heads
ff_dim = 128  # Size of feedforward network in transformer
num_classes = 8  # Number of output classes (hitobject types)

# Building the model
model = ModelWithMasking(maxlen_input, embed_dim, num_heads, ff_dim, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

# Example training
history = model.fit(padded_inputs, padded_outputs, epochs=10, batch_size=8)

# Generating prediction for the padded input at index 4
# output_for_input_4 = model.predict(padded_inputs[4:5])  # Predict for a batch of size 1
# print(output_for_input_4)


In [ ]:
predicted_output = model.predict(padded_inputs[2:3])  # Predict for a batch of size 1
# predicted_output will be of shape (1, time_steps, num_output_features), we remove the batch dimension to print it
predicted_output = np.squeeze(predicted_output, axis=0)  # Shape: (time_steps, num_output_features)

predicted_classes = np.argmax(predicted_output, axis=-1)

# Convert to one-hot encoding (hardmax transformation)
one_hot_predictions = np.zeros_like(predicted_output)

# Set the max probability class index to 1, others remain 0
for i, class_idx in enumerate(predicted_classes):
    one_hot_predictions[i, class_idx] = 1


for p in one_hot_predictions:
    print(p)

In [ ]:
history = model.fit(padded_inputs, padded_outputs, epochs=10, batch_size=8)

In [ ]:
# Assuming the model is trained and padded_inputs[0] is your test input
# padded_inputs[0]: shape (time_steps, num_input_features)
test_input = np.expand_dims(padded_inputs[20], axis=0)  # Add batch dimension, shape becomes (1, time_steps, 24)

# Get the model prediction
predicted_output = model.predict(test_input)

# predicted_output will be of shape (1, time_steps, num_output_features), we remove the batch dimension to print it
predicted_output = np.squeeze(predicted_output, axis=0)  # Shape: (time_steps, num_output_features)

predicted_classes = np.argmax(predicted_output, axis=-1)

# Convert to one-hot encoding (hardmax transformation)
one_hot_predictions = np.zeros_like(predicted_output)

# Set the max probability class index to 1, others remain 0
for i, class_idx in enumerate(predicted_classes):
    one_hot_predictions[i, class_idx] = 1


for p in one_hot_predictions:
    print(p)